In [1]:
import os
import numpy as np
import xarray as xr
import rioxarray
import zarr
from numcodecs import Blosc
import datetime
import glob
import re
import gc
import rasterio.errors

In [2]:
start_year = 2017
end_year = 2025
# now = datetime.datetime.now()
# end_year = np.datetime64(now).astype("datetime64[Y]").astype(int) + 1970
# print(end_year)

years = np.arange(start_year, end_year + 1, dtype="int32")
zarr_path = "../../../eodc/products/eodc/clms_vegetation_total_productivity/CLMS.zarr"

In [3]:
# Create zarr store and compressor
store = zarr.storage.LocalStore(zarr_path)
compressor = zarr.codecs.BloscCodec()

# create or open zarr folder
root = zarr.group(store=store)

#create or open dataset folder
dataset = "VPP"
ds = root.require_group(dataset)

# set extent (Austria)
res = 10

x0 = 4200005
y0 = 2500005

x_extent = np.arange(x0, 4900005, res)
y_extent = np.arange(y0, 3000005, res)


season1 = "SEASON1"
season2 = "SEASON2"

In [4]:
#create all arrays, if they don't exist

time_array = ds.require_array(
    name="time",
    shape=(len(years),),
    dtype="int32",
    chunks=(len(years),),
    dimension_names=["time"],
)
if np.all(time_array[:] == 0):
    time_array[:] = years

x_array = ds.require_array(
    name="x",
    shape=x_extent.shape,
    dtype="int32",
    chunks=(len(x_extent),),
    dimension_names=["x"]
)
if np.all(x_array[:] == 0):
    x_array[:] = x_extent

y_array = ds.require_array(
    name="y",
    shape=y_extent.shape,
    dtype="int32",
    chunks=(len(y_extent),),
    dimension_names=["y"]
)
if np.all(y_array[:] == 0):
    y_array[:] = y_extent


In [5]:
# Values for all parameters

fill_value_SPROD = 65535
scale_factor_SPROD = 0.1

attributes_SPROD={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": 0.1,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_SPROD,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 1095",
        "add_offset": 0.0,
        "long_name": "Season productivity (Small integral)",
    }


fill_value_RSLOPE = -32768
scale_factor_RSLOPE = 0.0001

attributes_RSLOPE={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_RSLOPE,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_RSLOPE,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Left slope (derivative)",
    }

fill_value_QFLAG = 0
scale_factor_QFLAG = 1.0

attributes_QFLAG={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_QFLAG,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_QFLAG,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "Flag_value": "(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",
        "add_offset": 0.0,
        "long_name": "Vegetation and Phenological Production Quality Flage",
    }


fill_value_EOSD = 0
scale_factor_EOSD = 1.0

attributes_EOSD={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_EOSD,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_EOSD,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "-",
        "add_offset": 0.0,
        "long_name": "End of season date",
    }


fill_value_LSLOPE = -32768
scale_factor_LSLOPE = 0.0001

attributes_LSLOPE={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_LSLOPE,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_LSLOPE,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Left slope (derivative)",
    }


fill_value_SOSV = -32768
scale_factor_SOSV = 0.0001

attributes_SOSV={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_SOSV,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_SOSV,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Start of season value",
    }

fill_value_AMPL = -32768
scale_factor_AMPL = 0.0001

attributes_AMPL={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_AMPL,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_AMPL,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Seasonal amplitude",
    }

fill_value_MAXV = -32768
scale_factor_MAXV = 0.0001

attributes_MAXV={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_MAXV,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_MAXV,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Max of season value (value at peak)",
    }

fill_value_EOSV = -32768
scale_factor_EOSV = 0.0001

attributes_EOSV={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_EOSV,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_EOSV,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "End of season value",
    }

fill_value_SOSD = 0
scale_factor_SOSD = 1.0

attributes_SOSD={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_SOSD,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_SOSD,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "PhysRange": "-",
        "add_offset": 0.0,
        "long_name": "Start of season date",
    }

fill_value_MAXD = 0
scale_factor_MAXD = 1.0

attributes_MAXD={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_MAXD,
        "file_creation": "2023:05:14 08:42:35",
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_MAXD,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "PhysRange": "0 to 1095",
        "add_offset": 0.0,
        "long_name": " Max of season date (time at peak)",
    }

fill_value_TPROD = 65535
scale_factor_TPROD = 0.1

attributes_TPROD={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": 0.1,
        "file_creation": "2021:07:23 13:22:50",
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_TPROD,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 1095",
        "add_offset": 0.0,
        "long_name": "Total productivity (Large integral)",
    }


fill_value_LENGTH = 0
scale_factor_LENGTH = 1.0

attributes_LENGTH={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_LENGTH,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_LENGTH,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 1096",
        "add_offset": 0.0,
        "long_name": "Season Length"
    }

fill_value_MINV = -32768
scale_factor_MINV = 0.0001

attributes_MINV={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2021",
        "scale_factor": scale_factor_MINV,
        "input_time_window": "2016-10-01 to 2021-02-29",
        "_FillValue": fill_value_MINV,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.5",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Base Value (value at minimum)"
    }

In [6]:
zarr.consolidate_metadata(store)
parameters = ("LENGTH", "MINV", "MAXV", "MAXD", "SOSD", "QFLAG", "EOSV", "TPROD", "AMPL", "SOSV", "LSLOPE", "EOSD", "RSLOPE", "SPROD")
# parameters = ('MAXV')

/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:3361: UserWarning: Object at LENGTH_old is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:3361: UserWarning: Object at SEASON1 is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/api/asynchronous.py:227: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [7]:
x0 = x_extent[0]  # 4_200_000
y0 = y_extent[0]  # 2_500_000
res = 10          # 10 m pro Pixel


def write_season(season, path, data_array):


    if "1" in season:
        s = "s1"
    elif "2" in season:
        s = "s2"
    
    year_to_index = {year: i for i, year in enumerate(years)}
    
    for y in years:
        print(f"year = {y}")

        tif_files = sorted([
            f for f in glob.glob(path)
            if "100m" not in f and f"_{y}_" in f and f"_{s}_" in os.path.basename(f)
        ])
            
        for filepath in tif_files:

            print(f"file: {filepath}")
            
            try:
                file = rioxarray.open_rasterio(filepath)
                tp = file.values.astype("float32")
            except Exception as e:
                print(f"Unexpected error with file: {filepath}")
                print(f"Error: {e}")
                continue


            match = re.search(r'E(\d+)N(\d+)', filepath)
            if match:
                # 1) Kachelecken (untere linke Ecke) in Meter
                xmin_corner = int(match.group(1)) * 100000  # z.B. 42 → 4_200_000
                ymin_corner = int(match.group(2)) * 100000

                # 2) Erste Zellmitte in dieser Kachel (wie im TIF)
                xmin_center = xmin_corner + res / 2       # 4_200_000 + 5 = 4_200_005
                ymin_center = ymin_corner + res / 2
            else:
                print("No match found.")
                continue

            ny_tile = tp.shape[1]   # Pixel in y
            nx_tile = tp.shape[2]   # Pixel in x

            # 3) Letzte Zellmitte in der Kachel
            xmax_center = xmin_center + (nx_tile - 1) * res
            ymax_center = ymin_center + (ny_tile - 1) * res

            print(f"tile centers x: {xmin_center} .. {xmax_center}")
            print(f"tile centers y: {ymin_center} .. {ymax_center}")

            # 4) Koordinaten → Indizes
            ix_min = int((xmin_center - x0) // res)
            ix_max = ix_min + nx_tile        # slice-Ende ist exklusiv
            iy_min = int((ymin_center - y0) // res)
            iy_max = iy_min + ny_tile

            start_idx = year_to_index[y]
            end_idx   = start_idx + tp.shape[0]

            # 5) Schreiben
            tp = tp[:, ::-1, :]
            data_array[start_idx:end_idx, iy_min:iy_max, ix_min:ix_max] = tp


            del tp
            gc.collect()


In [8]:
parameters[0]

'LENGTH'

In [9]:
zarr.consolidate_metadata(store)
for p in parameters[1:2]:
    print(p)
    
    pt = ds.require_group(p)

    shape = (len(years), len(y_extent), len(x_extent))
    chunk_shape = (1, 10000, 10000)

    fv =  globals()[f"fill_value_{p}"]

    if -32768 <= fv < 0:
        dtype = "int16"
    elif 0 <= fv <= 65535:
        dtype = "uint16"
    
    path = f"../../../eodc/private/openeo_platform/zarr_nacho/VPP_{p}/*.tif"


    data_array = pt.require_array(
        name=season1,
        shape=shape,
        chunks=chunk_shape,
        dtype=dtype,
        fill_value = globals()[f"fill_value_{p}"],
        compressor=compressor,
        dimension_names=["time", "y", "x"],
        attributes=globals()[f"attributes_{p}"],
        overwrite=False
    )
    write_season(season = season1, path = path, data_array=data_array)

    data_array = pt.require_array(
        name=season2,
        shape=shape,
        chunks=chunk_shape,
        dtype=dtype,
        fill_value = globals()[f"fill_value_{p}"],
        compressor=compressor,
        dimension_names=["time", "y", "x"],
        attributes=globals()[f"attributes_{p}"],
        overwrite=False
    )
    write_season(season = season2, path = path, data_array=data_array)

    

/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:3361: UserWarning: Object at LENGTH_old is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:3361: UserWarning: Object at SEASON1 is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/api/asynchronous.py:227: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:1119: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(


MINV
year = 2017
year = 2018
year = 2019
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N25-03035-010m_V101_s1_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2500005.0 .. 2599995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N26-03035-010m_V101_s1_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2600005.0 .. 2699995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N27-03035-010m_V101_s1_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2700005.0 .. 2799995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N28-03035-010m_V101_s1_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2800005.0 .. 2899995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E43N25-03035-010m_V101_s1_MINV.tif
tile centers x: 4300005.0 .. 4399995.0
tile centers y: 2500005.0 .. 2599995.0
file: ../../..

/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:1119: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(


year = 2017
year = 2018
year = 2019
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N25-03035-010m_V101_s2_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2500005.0 .. 2599995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N26-03035-010m_V101_s2_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2600005.0 .. 2699995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N27-03035-010m_V101_s2_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2700005.0 .. 2799995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E42N28-03035-010m_V101_s2_MINV.tif
tile centers x: 4200005.0 .. 4299995.0
tile centers y: 2800005.0 .. 2899995.0
file: ../../../eodc/private/openeo_platform/zarr_nacho/VPP_MINV/VPP_2019_S2_E43N25-03035-010m_V101_s2_MINV.tif
tile centers x: 4300005.0 .. 4399995.0
tile centers y: 2500005.0 .. 2599995.0
file: ../../../eodc